# Custom Graph Agent

This notebook demonstrates how to build a custom graph-based agent using LangGraph, showcasing advanced workflow control and state management, which we will use for multi-agent systems.

This notebook is divided into four main sections:

1. **Setup and Configuration**: 
   - Installation of required packages (langchain-openai, langchain-community, langgraph)
   - Configuration of OpenAI and Tavily API keys
   - Import of necessary typing and LangChain components
   - Langchain API key for tracing / observability

2. **Graph Construction**: 
   - Creation of a custom workflow using StateGraph
   - Visualize the graph using IPython display
   - Implementation of tool nodes and decision routing
   - Definition of model interaction functions
   - Setup of conditional edges for workflow control
   - Thread-based conversation tracking

3. **Interactive Implementation**:
   - Interactive chat loop

## Setup and Configuration

In [ ]:
!pip install langchain-openai==0.2.0
!pip install langchain-community==0.3.1
!pip install langgraph==0.2.23

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAi Key here:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily Key here:")

In [ ]:
# langchain key for langsmith tracing / observability
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Langchain Key here:")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ODSC_Workshop"

## Graph Construction

In [3]:
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

- The code initializes a search tool with TavilySearchResults limiting to 2 results, and creates a ChatOpenAI instance with zero temperature for consistent outputs

- `tool_node = ToolNode(tools)` creates a node that can handle the execution of the defined tools in the workflow

- `should_continue` function acts as a router - it examines the last message in the state and decides whether to use tools or end the conversation

- `call_model` function handles the actual interaction with the language model, taking the current state's messages and returning a new message list

- The `StateGraph` initialization creates a workflow framework that maintains conversation state between interactions

- Two main nodes are added to the workflow:
 - "agent": handles model interactions through call_model
 - "tools": manages external tool operations through tool_node

- The workflow is structured with specific edges:
 - START → agent: Initial entry point
 - agent → conditional routing (tools or END)
 - tools → agent: Returns control to agent after tool use

- `MemorySaver` is used to persist state between different runs of the graph, maintaining conversation context

- The graph is compiled into a LangChain Runnable, making it executable with standard LangChain interfaces

- The final execution shows how to invoke the workflow with an initial human message, using a specific thread_id for tracking

In [ ]:
tool = TavilySearchResults(max_results=2)
tools= [tool]

tool_node = ToolNode(tools)

model = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(tools)

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="What is the buy / sell recommendation for NVIDIA stock on December 4 2024.")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

Displaying the graph using ipython display.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)

**State Management and Thread Continuity**

- This code demonstrates conversation persistence by reusing the same thread_id (42), allowing the agent to maintain context from previous interactions

- The app.invoke call introduces a new question about AMD while retaining the context from the previous NVIDIA query

- By using the same thread_id in the config, the MemorySaver ensures all previous messages and context are available for this new interaction

In [ ]:
# Now when we pass the same "thread_id", the conversation context is retained
# via the saved state (i.e. stored list of messages)

final_state = app.invoke(
    {"messages": [HumanMessage(content="What about AMD?")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

In [6]:
config = {"configurable": {"thread_id": 42}}

## Loop the agent for interactive chat

**Interactive Chat Loop Implementation**

- Sets up an infinite loop that continuously accepts user input until specific exit commands ("quit", "exit", "q") are given

- Each user input is processed through the app.invoke method, maintaining the conversation state using the previously defined configuration

In [ ]:
from pprint import pprint
while True:
    user_input = input("User: ")
    # print("User: " + user_input)
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Assistant: Goodbye!")
        break
    final_state = app.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config
    )
    print()
    pprint(final_state)
    print()
    print("Assistant: " + final_state["messages"][-1].content)
    print()

### Practice Exercise 2

Replace the web search tool with RAG capabilities (using example from 3_1_multi_agent_system.ipynb)

In [ ]:
# Step 1. Copy over RAG code from 3_1_multi_agent_system.ipynb

# Step 2. Replace the web search tool with RAG tool

# Step 3. Test the new pipeline with a comparative analysis between two stocks of your choice.